## Initial Setup

1. Download the Citadel version 1.3.5.1 reference sample from [MalwareBazaar](https://bazaar.abuse.ch/sample/aa84cc4fd2150d0dd0986943e8567dca381c47c69b29673e8ba120d922fcced3/).
2. Unpack the sample with [UNPACME](https://www.unpac.me/results/c5afe5a6-c9e0-4e57-a86f-1160a5bf1214#/).

In [1]:
# Install required Python modules
import sys
!{sys.executable} -m pip install pefile
!{sys.executable} -m pip install pycryptodome

# Import required Python modules
import binascii
import hashlib
import pprint
import re
import struct

import pefile
from Crypto.Cipher import AES

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Load the unpacked sample
fp = open("aa84cc4fd2150d0dd0986943e8567dca381c47c69b29673e8ba120d922fcced3.dmp", "rb")
sample = fp.read()
fp.close()

pe = pefile.PE(data=sample)

## Decrypt Strings

Pseudocode:

    int __usercall CryptedStrings::_getA@<eax>(unsigned __int16 id@<ax>, int buffer@<edi>)
    {
      int *v2; // eax
      int i; // ecx
      int v4; // esi
      char v5; // dl
      int result; // eax

      v2 = &dword_404D10[2 * id];
      for ( i = 0; i < *(v2 + 1); *(v4 + buffer) = v5 )
      {
        v4 = i;
        v5 = i ^ *v2 ^ *(v2[1] + i);
        ++i;
      }
      result = *(v2 + 1);
      *(result + buffer) = 0;
      return result;
    }

Disassembly:

    .text:00417527                               CryptedStrings___getA proc near
    .text:00417527
    .text:00417527 0F B7 C0                                      movzx   eax, ax
    .text:0041752A 8D 04 C5 10 4D 40 00                          lea     eax, dword_404D10[eax*8]
    .text:00417531 33 D2                                         xor     edx, edx
    .text:00417533 33 C9                                         xor     ecx, ecx
    .text:00417535 66 3B 50 02                                   cmp     dx, [eax+2]
    .text:00417539 73 19                                         jnb     short loc_417554
    .text:0041753B 56                                            push    esi
    .text:0041753C
    .text:0041753C                               loc_41753C:
    .text:0041753C 8B 50 04                                      mov     edx, [eax+4]
    .text:0041753F 0F B7 F1                                      movzx   esi, cx
    .text:00417542 8A 14 32                                      mov     dl, [edx+esi]
    .text:00417545 32 10                                         xor     dl, [eax]
    .text:00417547 32 D1                                         xor     dl, cl
    .text:00417549 41                                            inc     ecx
    .text:0041754A 88 14 3E                                      mov     [esi+edi], dl
    .text:0041754D 66 3B 48 02                                   cmp     cx, [eax+2]
    .text:00417551 72 E9                                         jb      short loc_41753C
    .text:00417553 5E                                            pop     esi
    .text:00417554
    .text:00417554                               loc_417554:
    .text:00417554 0F B7 40 02                                   movzx   eax, word ptr [eax+2]
    .text:00417558 C6 04 38 00                                   mov     byte ptr [eax+edi], 0
    .text:0041755C C3                                            retn
    .text:0041755C                               CryptedStrings___getA endp

In [3]:
def decrypt_string(pe, id_):
    match = re.search(rb'\x8d\x04\xc5(?P<enc_strings_addr>[\s\S]{4})', 
        pe.get_memory_mapped_image())
    enc_strings_addr = struct.unpack("I", match.group("enc_strings_addr"))[0]
    
    enc_string_addr = enc_strings_addr + 8 * id_
    enc_string_rva = enc_string_addr - pe.OPTIONAL_HEADER.ImageBase
    
    xor_key = pe.get_word_at_rva(enc_string_rva)
    encbuf_len = pe.get_word_at_rva(enc_string_rva + 2)
    encbuf_addr = pe.get_dword_at_rva(enc_string_rva + 4)
    encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
    encbuf = pe.get_data(encbuf_rva, encbuf_len)
    
    plainbuf = b""
    for i in range(len(encbuf)):
        pb = (i ^ xor_key ^ encbuf[i]) & 0xff
        plainbuf += struct.pack("B", pb)
        
    return plainbuf.decode()


for i in range(311):
    plain_string = decrypt_string(pe, i)
    print(f"{i}: {plain_string}")

0: Module: %u
Type: %s
Title: %s
Info: %s

1: ERROR
2: FAILURE
3: SUCCESS
4: UNEXPECTED
5: UNKNOWN
6: rurl
7: surl
8: furl
9: uid
10: mask
11: post
12: extensions
13: rules
14: patterns
15: %tokenspy%
16: url
17: buid
18: ruid
19: puid
20: session
21: data
22: get_status
23: status
24: status_cache_time
25: Can't compile tokenspy rules.
26: fileName=[%S], fileSize=[%u], fileCRC32=[0x%08X].
27: set_url
28: data_before

29: data_inject

30: data_after

31: data_end

32: %webinject%
33: Can't compile webinjects.
34: fileName=[%S], fileSize=[%u], fileCRC32=[0x%08X], processedInjects=[%u].
35: Webinjects has been compiled.
36: result=[%u], fileName=[%S], fileSize=[%u], fileCRC32=[0x%08X], processedInjects=[%u].
37: *vmware*
38: *sandbox*
39: *virtualbox*
40: *geswall*
41: *bufferzone*
42: *safespace*
43: *.ru
44: *.con.ua
45: *.by
46: *.kz
47: cmd.exe
48: powershell.exe
49: 
exit

50: 
prompt $Q$Q$Q$Q$Q$Q$Q$Q$Q$Q[ $P ]$G

51: screenshots\%s\%04x_%08x.jpg
52: unknown
53: image/jpeg
54: Softw

## Extract Version

Pseudocode:

    ...
    version = 0x1030501;
    r = BinStorage::_addItem(binStorage, 4u, 10003, 0x20000, &version);
    ...
    
Disassembly:

    ...
    .text:00425F9A 8D 44 24 0C                                   lea     eax, [esp+4D8h+version]
    .text:00425F9E 50                                            push    eax
    .text:00425F9F 57                                            push    edi
    .text:00425FA0 68 13 27 00 00                                push    2713h
    .text:00425FA5 6A 04                                         push    4
    .text:00425FA7 5B                                            pop     ebx
    .text:00425FA8 8B C6                                         mov     eax, esi
    .text:00425FAA C7 44 24 18 01 05 03 01                       mov     [esp+4E4h+version], 1030501h
    .text:00425FB2 E8 3B DA 00 00                                call    BinStorage___addItem
    ...

In [4]:
match = re.search(rb'\x68\x13\x27\x00\x00[\s\S]{1,8}?\xc7\x44\x24[\s\S](?P<version>[\s\S]{4})', 
    pe.get_memory_mapped_image())
version_bytes = match.group("version")
version = ".".join([str(b) for b in version_bytes[::-1]])
print(version)

1.3.5.1


## Decrypt BaseConfig

Pseudocode:

    _BYTE *__usercall Core::getBaseConfig@<eax>(void *bc@<eax>)
    {
      _BYTE *result; // eax
      int v2; // edx
      int v3; // esi
      int v4; // ecx

      Mem::_copy(bc, dword_402780, 1452u);
      v3 = v2;
      v4 = dword_437168 + g_self_hdl - result;
      do
      {
        *result ^= result[v4];
        ++result;
        --v3;
      }
      while ( v3 );
      return result;
    }

Disassembly:

    .text:00419601                               Core__getBaseConfig proc near
    .text:00419601
    .text:00419601 56                                            push    esi
    .text:00419602 BA AC 05 00 00                                mov     edx, 5ACh
    .text:00419607 52                                            push    edx
    .text:00419608 68 80 27 40 00                                push    offset dword_402780
    .text:0041960D 50                                            push    eax
    .text:0041960E E8 AE 2A 01 00                                call    Mem___copy
    .text:00419613 8B 0D 74 6D 43 00                             mov     ecx, g_self_hdl
    .text:00419619 03 0D 68 71 43 00                             add     ecx, dword_437168
    .text:0041961F 8B F2                                         mov     esi, edx
    .text:00419621 2B C8                                         sub     ecx, eax
    .text:00419623
    .text:00419623                               loc_419623:
    .text:00419623 8A 14 01                                      mov     dl, [ecx+eax]
    .text:00419626 30 10                                         xor     [eax], dl
    .text:00419628 40                                            inc     eax
    .text:00419629 4E                                            dec     esi
    .text:0041962A 75 F7                                         jnz     short loc_419623
    .text:0041962C 5E                                            pop     esi
    .text:0041962D C3                                            retn
    .text:0041962D                               Core__getBaseConfig endp

In [5]:
match = re.search(rb'\xba(?P<encbuf_len>[\s\S]{2}\x00\x00)\x52\x68(?P<encbuf_addr>[\s\S]{4})',
    pe.get_memory_mapped_image())

encbuf_len = struct.unpack("I", match.group("encbuf_len"))[0]
encbuf_addr = struct.unpack("I", match.group("encbuf_addr"))[0]
encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
encbuf = pe.get_data(encbuf_rva, encbuf_len)

# #define PESECTION_OF_BASECONFIG_KEY 2
section = pe.sections[2]
xor_key = pe.get_data(section.VirtualAddress, encbuf_len)

base_config = b""
for i, eb in enumerate(encbuf):
    pb = eb ^ xor_key[i % len(xor_key)]
    base_config += struct.pack("B", pb)
    
print(base_config)

b'\x16\xaf\x81\xe7\x13\xa08f\xec[{\x87\x0c\xab\xb5\x97\xe2Z]\xc0?\xff|}\xff\x17iQ\xebW\xd7\xb9\xdeA\x12_\'\xf6\x86\x90/K\x81\xf0\\\x1e:m4\xeb\x11q\xc6\x9a\x18\\E\x9b\xd31\xf75<\x8fM\x1b\n\xe4\x18\x92\x00\x00L\xed\xb0!\x95\x11\x80>\xd3\xd8\xef\xe3J\x7f\x18Q\x8af\xa9\x0cQg\xe6\xd6\xadf\xf1\xdda)H4\t\x00\x04\x00\x82+\x90\xf4\x05J\x80\x7f\x92U\x83C\x14\x9fsJ\xc2\x94\xddtDIM\xb4\x17X\xcc|2\xea\xa6v\xea\x15qu\x13iJ\xabh\xc5b\x1c-o\xbaR\x12d*S[\xed\xa8\xd0<\x00\x00\x00\x8c\x94~l\x94w+\xabE\\\xd7\xd0\x92w\xbc\xc3\x1b\xfe\x04\x8e\t\x81\xd4Y9z\x89\x99\x9e%B\x84\xe0\x91\x9e\xc8I\xf7\xc7|\x06\x00\x03\x00\xd9\x12\x99 &s\xdcen\x90\xe3p(\x17yw\xfe\x13C\x13v\t\x8e\xf3\xd1@L\xd24`X\xa6\x05?\x02\x1b\x98\xf7\xc7?\x0b<`\x7fn\xec\xff\xd2\'\xe8\xd54\x95#\xd7V\xf4\xf8\xc2\x94\x81\xc1\\\'\xde\xa5\xfd\x9c\xb7\xb1O\x84\xa6*RnT"\xa3\x12\x08\x00\x04\x00\x87\x07\x00Xz\xc81\xaa\x12\x96CA+\xed\x95\x8b\xcd\xca\xfb\x9d\x08\x05\xc1%"\xe1\xed\x9f\xf5\xe2\xc5\xcc\x92\xde\x91`*W\xa2\xfb/\x08\x93[\xb6\xbe\xcf\xd0 iS\xf0\xf

## Parse BaseConfig

In [6]:
flags = struct.unpack("I", base_config[0x44:0x44+4])[0]
print(f"flags: 0x{flags:x}\n")

delay_cmd_list1 = struct.unpack("I", base_config[0x68:0x68+4])[0]
print(f"delay_cmd_list1: {delay_cmd_list1}\n")

delay_cmd_list2 = struct.unpack("I", base_config[0xa4:0xa4+4])[0]
print(f"delay_cmd_list2: {delay_cmd_list2}\n")

delayStats = struct.unpack("I", base_config[0xd0:0xd0+4])[0]
print(f"delayStats: {delayStats}\n")

delayReport = struct.unpack("I", base_config[0x124:0x124+4])[0]
print(f"delayReport: {delayReport}\n")

defaultConfig2 = base_config[0x163:0x163+101].split(b"\x00")[0].decode().replace("http", "hxxp")
print(f"defaultConfig2: {defaultConfig2}\n")

defaultConfig = base_config[0x260:0x260+101].split(b"\x00")[0].decode().replace("http", "hxxp")
print(f"defaultConfig: {defaultConfig}\n")

delay_modules = struct.unpack("I", base_config[0x2dc:0x2dc+4])[0]
print(f"delay_modules: {delay_modules}\n")

baseKey = base_config[0x30a:0x30a+258]
print(f"baseKey: {baseKey}\n")

defaultConfig3 = base_config[0x423:0x423+101].split(b"\x00")[0].decode().replace("http", "hxxp")
print(f"defaultConfig3: {defaultConfig3}\n")

defaultBotnet = base_config[0x4e6:0x4e6+21].split(b"\x00\x00")[0].decode()
print(f"defaultBotnet: {defaultBotnet}\n")

delay_last_version_url = struct.unpack("I", base_config[0x574:0x574+4])[0]
print(f"delay_last_version_url: {delay_last_version_url}\n")

flags: 0x9218

delay_cmd_list1: 262153

delay_cmd_list2: 60

delayStats: 196614

delayReport: 262152

defaultConfig2: hxxp://reserve-wadergroup.biz/images/cross/file.php|file=config.dll

defaultConfig: hxxp://ohajaonha.in/wp-includes/css/xfawiz/php/file.php|file=config.dll

delay_modules: 65540

baseKey: b'Pk\xa6\xafG!/e\x08\x0c\x05\x17\xc4\x82\x11L\xea\xa5\x883z\xb8\x8c\xb6\xaf\x95\x95\x02\xf5,W\x11)\xd5\xc8\x8a\xa2[\xae\x01\xe5}xzg&\';\xc8h\x01`\xa7_\x9b8?|\x82\xe3\xb5\xd4\xec\xea\xce\x83\xab\xda\xf2*qf;ND%\x98\xac\x92\x95<4\xc3\x96a\xba\xa5\xa6>\x87}\xd1m\x85Kpe\xb0\x0f\x0c(\xf8,\x04I\x04\x9a\x9b Y\xd4!\x7f\x97L\xd3\xfc:\xbb\x8b\\X\xd7\x1fvfG\xaa\x06\r\xf5\xf8W\xa0\xee\xddQ\xc1\xcd\xc8\xcb\xbc8\xfb\xb3R\xa2\x10\xd9\x16\x10\xf9\x9d\xc2\x0bbCT\xa3\xc0\x8c\xf0\x8e\xd4\xe7q\x7f\x1eZ\xd6\xc79\x9e\xc7\x12X{\xb7%U\xfd\x8f\x9e\x13\xecB\x94?\xe23\x86\x8d\xf1\x12\xe8W\xbbO\xad\xe0j\xeb\xba\x80Hr\xb3\xa1\x90/\x03\xd5\xe3\x1e\x87\x16\xfa\xbf\xb7q\xdaF\xd8\xd9\x815\xc4\x05t\x19\xc2B\x1b"\xfd\xf0

## Download DynamicConfig

![](request.png)

There are a number of keys and custom crypto components involved in decrypting C2 requests and responses.

## Extract Login Key

Pseudocode:

    ...
    r = BinStorage::_addItemAsUtf8StringA("C1F20D2340B519056A7D89B7DF4B0FFF", binStorage, 10021);
    ...

Disassembly:

    ...
    .text:00425F53 68 25 27 00 00                                push    2725h
    .text:00425F58 56                                            push    esi
    .text:00425F59 B9 58 1C 40 00                                mov     ecx, offset aC1f20d2340b519
    .text:00425F5E E8 7A DB 00 00                                call    BinStorage___addItemAsUtf8StringA
    ...

In [7]:
match = re.search(rb'\x68\x25\x27\x00\x00[\s\S]\xb9(?P<login_key_addr>[\s\S]{4})', 
    pe.get_memory_mapped_image())

login_key_addr = struct.unpack("I", match.group("login_key_addr"))[0]
login_key_rva = login_key_addr - pe.OPTIONAL_HEADER.ImageBase
login_key = pe.get_data(login_key_rva, 32)
print(f"{login_key=}")

login_key=b'C1F20D2340B519056A7D89B7DF4B0FFF'


## Generate AES Key

Pseudocode:

    ...
    Core::getBaseConfig(&base_config);
    login_key_len = Str::_LengthA("C1F20D2340B519056A7D89B7DF4B0FFF");
    Crypt::_md5Hash(login_key_md5_digest, v3, login_key_len);
    Crypt::_rc4_w_login_key_xor(v4, &base_config.baseKey, login_key_md5_digest, 0x10u);
    Mem::_copy(aes_key, login_key_md5_digest, 0x10u);
    ...

In [8]:
def rc4_keystate(S, in_buf):
    out_buf = b""

    i = S[256]
    j = S[257]
    
    for in_byte in in_buf:
        (i, j, S, K) = rc4_prga(i, j, S)
        out_byte = in_byte ^ K
        out_buf += struct.pack("B", out_byte)

    return out_buf


def rc4_prga(i, j, S):
    i = (i + 1) % 256
    j = (j + S[i]) % 256

    S = bytearray(S)
    S[i], S[j] = S[j], S[i]
    S = bytes(S)

    K = S[(S[i] + S[j]) % 256]

    return (i, j, S, K)


def xor(inbuf, key):
    outbuf = b""
    for i, ib in enumerate(inbuf):
        ob = ib ^ key[i % len(key)]
        outbuf += struct.pack("B", ob)
        
    return outbuf


md5 = hashlib.md5()
md5.update(login_key)
round1 = rc4_keystate(baseKey, md5.digest())
aes_key = xor(round1, login_key)
print(f"{aes_key=}")

aes_key=b'\\\xc8\xc6\xf81\xef6\x04\x1a\xe6\xc5\xbaf\xf9\xd5$'


## Generate Custom RC4 Keystate For Requests

The AES key is used with a custom RC4 key scheduling algorithm (KSA) to generate an RC4 keystate that is used for request encryption.

Pseudocode:

    int __userpurge Crypt::_rc4Init_custom@<eax>(int result@<eax>, int a2, int a3)
    {
      int v3; // ecx
      _BYTE *v4; // edx
      char *v5; // esi
      int v6; // edi
      char v7; // dl
      unsigned int v8; // esi
      char v9; // dl
      char v10; // cl
      char v11; // dl
      unsigned __int8 v12; // [esp+Ah] [ebp-2h]
      unsigned __int8 v13; // [esp+Bh] [ebp-1h]

      v3 = 0;
      v13 = 0;
      v12 = 0;
      *(result + 256) = 0;
      *(result + 258) = 0;
      v4 = result;
      do
        *v4++ = v3++;
      while ( v3 < 0x100u );
      v5 = result;
      v6 = 256;
      do
      {
        v7 = *v5;
        v12 += *v5 + *(v13++ + a2);
        *v5 = *(v12 + result);
        *(v12 + result) = v7;
        if ( v13 == a3 )
          v13 = 0;
        ++v5;
        --v6;
      }
      while ( v6 );
      v8 = 0;
      a3 = 0xD6490B8F;
      do
      {
        v9 = *(&a3 + v6) & 7;
        v10 = *(&a3 + v6++) >> 3;
        if ( v6 == 4 )
          v6 = 0;
        switch ( v9 )
        {
          case 0:
            *(v8 + result) = ~*(v8 + result);
            break;
          case 1:
            *(v8 + result) ^= v10;
            break;
          case 2:
            *(v8 + result) += v10;
            break;
          case 3:
            *(v8 + result) -= v10;
            break;
          case 4:
            v11 = __ROR1__(*(v8 + result), v10);
            goto LABEL_16;
          case 5:
            v11 = __ROL1__(*(v8 + result), v10);
    LABEL_16:
            *(v8 + result) = v11;
            break;
          case 6:
            ++*(v8 + result);
            break;
          case 7:
            --*(v8 + result);
            break;
          default:
            break;
        }
        ++v8;
      }
      while ( v8 < 0x100 );
      return result;
    }

Disassembly:

    ...
    .text:0042D8A2 33 F6                                         xor     esi, esi
    .text:0042D8A4 C7 45 0C 8F 0B 49 D6                          mov     [ebp+arg_4], 0D6490B8Fh
    ...

In [9]:
def get_comms_rc4_key(key, salt):
    S = []
    for i in range(256):
        S.append(i)

    S.append(0)
    S.append(0)

    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % len(key)]) % 256
        S[i], S[j] = S[j], S[i]

    for i in range(256):
        v9 = salt[i % len(salt)] & 7
        v10 = salt[i % len(salt)] >> 3

        if v9 == 0:
            S[i] = (~S[i]) & 0xff

        if v9 == 1:
            S[i] = (S[i] ^ v10) & 0xff

        if v9 == 2:
            S[i] = (S[i] + v10) & 0xff

        if v9 == 3:
            S[i] = (S[i] - v10) & 0xff

        if v9 == 4:
            S[i] = self.ror(S[i], v10, 32) & 0xff

        if v9 == 5:
            S[i] = self.rol(S[i], v10, 32) & 0xff

        if v9 == 6:
            S[i] = (S[i] + 1) & 0xff

        if v9 == 7:
            S[i] = (S[i] - 1) & 0xff

    return bytes(S)


match = re.search(rb'\x33\xf6\xc7\x45\x0c(?P<rc4_ksa_salt>[\s\S]{4})', pe.get_memory_mapped_image())
rc4_ksa_salt = struct.unpack("I", match.group("rc4_ksa_salt"))[0]
print(f"rc4_ksa_salt=0x{rc4_ksa_salt:x}")

request_rc4_keystate = get_comms_rc4_key(aes_key, struct.pack("I", rc4_ksa_salt))
print(f"{request_rc4_keystate=}")

rc4_ksa_salt=0xd6490b8f
request_rc4_keystate=b'2$\x06\xc6\x1c(\xff\x07KD1\xfan\xb0M\x8c\xbe\xc3\x95l\x8b\xb3\xc4X\x88\xd2\x19\x17\xbd\xfd\xe6z\x08z[Z\x1dA\xf3\x9f\xb9#GK\xf7\x1b\x18`\xb2gA\xec\xc0\x17\r\xe5Z\x9f{)\xbc\xad\xf9y\xe0\xba.a\\\xd9\xf5\xd1\xd4O\x02\x81yW\x05\x06}\x02\x03\xa0\xa7\xa0\xee\xf4\x83=\x8e\xe0\xed1m!\x98\xa9\xdb\xee\xc1\xe4y_\xdc\x80\x94u\xf1%\xab\x80\x82\xbbx\x08H\xa5\'\xb0\x81\x00\x84\x0e\xd0\xf0\xc5c.\x93%w\xdd\xb13::m\xa5\x18\xf3\x13<\x91/\xd3\x1a\xb8\xe8\x01\xb0\xe1L\r8\xe4\x18\xa2t\x16\x07e\xe3H\xa4\xb5\xd0\xdd\x96\xb4O@\xc76\\\x87lt\xae5\x1e\x95],Y\xcd\x13\xe7\xce\x92j\xf8\xbf)~\x89i\xe1\xf6\xcbr\xd7B\x86`R\x92\x8f\xa8\xe7l\xc4\xda\xb7n7\xc6\x8e\xdeW\xaa\xc5(\xca"\xeb\x12D\xfa\xac\x9c\x13{\xaf\x93]\x89\xf4\xf4>;!\xc2.?h\x91\xd7\x91\xffF\x92kPH\xa5\x00\x00'


## Decrypt DynamicConfig Request

In [10]:
def visual_decrypt(enc_buf):
    buf = bytearray(enc_buf)

    for i in range(len(buf)-1, 0, -1):
        buf[i] ^= buf[i-1]

    plain_buf = bytes(buf)

    return plain_buf


# from PCAP
post_data = binascii.unhexlify("6fc03dcbe7928fda4e6fc4066a38d49c13e6c60a399e34207b322b27d08289fa8b1a0467ac966fced1249ecb031a9db36d04cdedf07c784f5ed398b5f8035241502506dcff706005da520633ff240c2b69f4feb7e5b59345e9f3b70aa669fc6dc116e53b1bbdf7e4074a7f9c1b8248e694f50fe316dd1fa41fc4")

round1 = rc4_keystate(request_rc4_keystate, post_data)
round2 = xor(round1, login_key)
request_binstorage = visual_decrypt(round2)

print(request_binstorage)

b"\x960\xc0:\xf0'\x13\x1a\xef\x82\xf6\xf46d\xd7\xb5\xa4\xcc\xb5\xb1z\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00m\xc8se\\>\x1e\x17\xa9L\xa8\x8b\xb8\x1b\xbc\xf6%'\x00\x00\x00\x00\x00\x00 \x00\x00\x00 \x00\x00\x00C1F20D2340B519056A7D89B7DF4B0FFF&'\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\n\x00\x00\x00config.dll"


## Parse DynamicConfig Request

In [11]:
def parse_binstorage_header(binstorage):
    header = {
        "randData": binstorage[0:20],
        "size": struct.unpack("I", binstorage[20:24])[0],
        "flags": struct.unpack("I", binstorage[24:28])[0],
        "count": struct.unpack("I", binstorage[28:32])[0],
        "md5Hash": binstorage[32:48],
    }
        
    return header


def parse_binstorage_sections(section_data, num_sections):
    CFGIDS = {
        20001: "CFGID_LAST_VERSION",
        20002: "CFGID_LAST_VERSION_URL",
        20003: "CFGID_URL_SERVER_0",
        20004: "CFGID_URL_ADV_SERVERS",
        20005: "CFGID_HTTP_FILTER",
        20006: "CFGID_HTTP_POSTDATA_FILTER",
        20007: "CFGID_HTTP_INJECTS_LIST",
        20008: "CFGID_DNS_LIST",
        20009: "CFGID_DNS_FILTER",
        20010: "CFGID_CMD_LIST",
        20011: "CFGID_HTTP_MAGICURI_LIST",
        20012: "CFGID_FILESEARCH_KEYWORDS",
        20013: "CFGID_FILESEARCH_EXCLUDES_NAME",
        20014: "CFGID_FILESEARCH_EXCLUDES_PATH",
        20015: "CFGID_KEYLOGGER_PROCESSES",
        20016: "CFGID_KEYLOGGER_TIME",
        20017: "CFGID_FILESEARCH_MINYEAR",
        20018: "CFGID_WEBINJECTS_URL",
        20019: "CFGID_TOKENSPY_URL",
        20020: "CFGID_HTTPVIP_URLS",
        20101: "CFGID_VIDEO_QUALITY",
        20102: "CFGID_VIDEO_LENGTH",
        20201: "CFGID_MONEY_PARSER_ENABLED",
        20202: "CFGID_MONEY_PARSER_INCLUDE",
        20203: "CFGID_MONEY_PARSER_EXCLUDE"    
    }
    
    sections = []
    offset = 0
    for i in range(num_sections):
        section = {}
        
        id_ = struct.unpack("I", section_data[offset:offset+4])[0]
        if int(id_) in CFGIDS:
            section["id"] = f'{CFGIDS[int(id_)]} ({id_})'
        else:
            section["id"] = id_
            
        section["flags"] = struct.unpack("I", section_data[offset+4:offset+8])[0]
        section["size"] = struct.unpack("I", section_data[offset+8:offset+12])[0]
        section["realSize"] = struct.unpack("I", section_data[offset+12:offset+16])[0]
        # additional parsing of data is left as an exercise
        section["data"] = section_data[offset+16:offset+16+section["size"]]
        
        offset += 4 + 4 + 4 + 4 + section["size"]
        
        sections.append(section)
        
    return sections


binstorage_header = parse_binstorage_header(request_binstorage)
print("binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(request_binstorage[48:], binstorage_header["count"])
print()
print("binstorage_sections:\n")
pprint.pprint(binstorage_sections)

binstorage_header:

{'count': 2,
 'flags': 0,
 'md5Hash': b'm\xc8se\\>\x1e\x17\xa9L\xa8\x8b\xb8\x1b\xbc\xf6',
 'randData': b"\x960\xc0:\xf0'\x13\x1a\xef\x82\xf6\xf46d\xd7\xb5"
             b'\xa4\xcc\xb5\xb1',
 'size': 122}

binstorage_sections:

[{'data': b'C1F20D2340B519056A7D89B7DF4B0FFF',
  'flags': 0,
  'id': 10021,
  'realSize': 32,
  'size': 32},
 {'data': b'config.dll', 'flags': 0, 'id': 10022, 'realSize': 10, 'size': 10}]


## Extract XOR Key For Responses

Pseudocode:

    ... 
    *a1 ^= 0x32C1A4FCu;
    v16 = a1[3];
    v17 = a1[2];
    *(a1 + 1) ^= 0xABC8F546;
    *(a1 + 2) ^= 0xDCCFC5D0;
    *(a1 + 3) ^= 0x42AB5073u;
    ... 

Disassembly:

    ... 
    .text:0042E1F6 81 30 FC A4 C1 32                             xor     dword ptr [eax], 32C1A4FCh
    .text:0042E1FC 0F B6 50 03                                   movzx   edx, byte ptr [eax+3]
    .text:0042E200 0F B6 78 02                                   movzx   edi, byte ptr [eax+2]
    .text:0042E204 81 70 04 46 F5 C8 AB                          xor     dword ptr [eax+4], 0ABC8F546h
    .text:0042E20B 81 70 08 D0 C5 CF DC                          xor     dword ptr [eax+8], 0DCCFC5D0h
    .text:0042E212 81 70 0C 73 50 AB 42                          xor     dword ptr [eax+0Ch], 42AB5073h
    ... 

In [12]:
match = re.search(rb'\x81\x30(?P<xor_1>[\s\S]{4})[\s\S]{8}\x81\x70[\s\S](?P<xor_2>[\s\S]{4})\x81\x70[\s\S](?P<xor_3>[\s\S]{4})\x81\x70[\s\S](?P<xor_4>[\s\S]{4})', pe.get_memory_mapped_image())

xor_1 = struct.unpack("I", match.group("xor_1"))[0]
print(f"xor_1=0x{xor_1:x}")
xor_2 = struct.unpack("I", match.group("xor_2"))[0]
print(f"xor_2=0x{xor_2:x}")
xor_3 = struct.unpack("I", match.group("xor_3"))[0]
print(f"xor_3=0x{xor_3:x}")
xor_4 = struct.unpack("I", match.group("xor_4"))[0]
print(f"xor_4=0x{xor_4:x}")

xor_key = match.group("xor_1") + match.group("xor_2") + match.group("xor_3") + match.group("xor_4")
print(f"{xor_key=}")

xor_1=0x32c1a4fc
xor_2=0xabc8f546
xor_3=0xdccfc5d0
xor_4=0x42ab5073
xor_key=b'\xfc\xa4\xc12F\xf5\xc8\xab\xd0\xc5\xcf\xdcsP\xabB'


## Decrypt DynamicConfig Response

Pseudocode:

    char __userpurge DynamicConfig::updateConfig@<al>(int md@<ebx>, char a2)
    {
      // [COLLAPSED LOCAL DECLARATIONS. PRESS KEYPAD CTRL-"+" TO EXPAND]

      Core::getBaseConfig(&base_config);
      login_key_len = Str::_LengthA("C1F20D2340B519056A7D89B7DF4B0FFF");
      Crypt::_md5Hash(login_key_md5_digest, v3, login_key_len);
      Crypt::_rc4_w_login_key_xor(v4, &base_config.baseKey, login_key_md5_digest, 0x10u);
      Mem::_copy(aes_key, login_key_md5_digest, 0x10u);
      v1 = *md;
      v14 = 0;
      if ( v1 )
      {
        v6 = BinStorage::_unpack(v1, *(md + 4), aes_key);
        *(md + 4) = v6;
    ...

    int __userpurge BinStorage::_unpack@<eax>(int data@<eax>, unsigned int data_size@<edi>, const void *aes_key)
    {
      // [COLLAPSED LOCAL DECLARATIONS. PRESS KEYPAD CTRL-"+" TO EXPAND]

      if ( data_size - 48 > 0x31FFFD0 )
        return 0;
      if ( aes_key )
      {
        Mem::_copy(aes_key_cp, aes_key, 0x10u);
        to_aes_decrypt(data, v4, data_size);
        if ( data_size )
        {
          v5 = data_size - 1;
          if ( data_size != 1 )
          {
            do
            {
              *(v5 + data) ^= *(v5 + data - 1);
              --v5;
            }
            while ( v5 );
          }
        }
      }
    ...

In [13]:
# Load encrypted DynamicConfig from PCAP
fp = open("config.dll", "rb")
config_dll = fp.read()
fp.close()

aes = AES.new(aes_key, AES.MODE_ECB)
round1 = aes.decrypt(config_dll)
round2 = xor(round1, xor_key)
response_binstorage = visual_decrypt(round2)

print(response_binstorage)

b'\xcaw\x0f\x9d\xaa\xbe0\x86M\xfdW0\xc6x\x14\nu\xa8S\xca\xf0\x14\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00f\x87\x94sj\x05\xd1\x85\xbf\x9d\xb5\xc7\xb7\xd7}\xd0!N\x00\x00\x00\x00\x00\x10\x04\x00\x00\x00\x04\x00\x00\x00\x01\x05\x03\x01"N\x00\x00\x00\x00\x00\x10E\x00\x00\x00E\x00\x00\x00http://ohajaonha.in/wp-includes/css/xfawiz/php/file.php|file=soft.exe#N\x00\x00\x00\x00\x00\x107\x00\x00\x007\x00\x00\x00http://ohajaonha.in/wp-includes/css/xfawiz/php/gate.php2N\x00\x00\x00\x00\x00\x107\x00\x00\x007\x00\x00\x00http://ohajaonha.in/wp-includes/css/xfawiz/php/file.php$N\x00\x00\x01\x00\x00\x10G\x00\x00\x00k\x00\x00\x00\xb5\xff\xff\xffhttp://reserve-host1/fold\x0e\xfe\xf6\xef\xbf\x06ile.php|\x08=con\x07g.bin\x00`*\x19\xc042\x00\x00\x00\x00\x00\x00\x00\x90\x00\xff*N\x00\x00\x00\x00\x00\x10C\x00\x00\x00C\x00\x00\x00hostname\x00tasklist\x00ipconfig /all\x00netsh firewall set opmode disable\x00\x00%N\x00\x00\x00\x00\x00\x108\x00\x00\x008\x00\x00\x00#*wellsfargo.com/*\x00@*payment.com/*\x00!http://*.

## Parse DynamicConfig

In [14]:
binstorage_header = parse_binstorage_header(response_binstorage)
print("binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(response_binstorage[48:], binstorage_header["count"])
print()
print("binstorage_sections:\n")
pprint.pprint(binstorage_sections)

binstorage_header:

{'count': 15,
 'flags': 0,
 'md5Hash': b'f\x87\x94sj\x05\xd1\x85\xbf\x9d\xb5\xc7\xb7\xd7}\xd0',
 'randData': b'\xcaw\x0f\x9d\xaa\xbe0\x86M\xfdW0\xc6x\x14\nu\xa8S\xca',
 'size': 5360}

binstorage_sections:

[{'data': b'\x01\x05\x03\x01',
  'flags': 268435456,
  'id': 'CFGID_LAST_VERSION (20001)',
  'realSize': 4,
  'size': 4},
 {'data': b'http://ohajaonha.in/wp-includes/css/xfawiz/php/file.php|file=sof'
          b't.exe',
  'flags': 268435456,
  'id': 'CFGID_LAST_VERSION_URL (20002)',
  'realSize': 69,
  'size': 69},
 {'data': b'http://ohajaonha.in/wp-includes/css/xfawiz/php/gate.php',
  'flags': 268435456,
  'id': 'CFGID_URL_SERVER_0 (20003)',
  'realSize': 55,
  'size': 55},
 {'data': b'http://ohajaonha.in/wp-includes/css/xfawiz/php/file.php',
  'flags': 268435456,
  'id': 'CFGID_WEBINJECTS_URL (20018)',
  'realSize': 55,
  'size': 55},
 {'data': b'\xb5\xff\xff\xffhttp://reserve-host1/fold\x0e\xfe\xf6\xef\xbf\x06i'
          b'le.php|\x08=con\x07g.bin\x00`*\x19\xc

## Commands

Pseudocode:

    ...
    while ( 1 )
    {
    CryptedStrings::_getW(word_4056D0[4 * v15], String2);
    if ( !lstrcmpiW(*v14, String2) )
      break;
    if ( ++v15 >= 40 )
      goto LABEL_33;
    }
    ...

Disassembly:

    ...
    .text:00414BBB 33 FF                                         xor     edi, edi
    .text:00414BBD
    .text:00414BBD                               loc_414BBD:
    .text:00414BBD 66 8B 04 FD D0 56 40 00                       mov     ax, ds:word_4056D0[edi*8]
    .text:00414BC5 8D B5 54 FD FF FF                             lea     esi, [ebp+String2]
    .text:00414BCB E8 8D 29 00 00                                call    CryptedStrings___getW
    .text:00414BD0 8B C6                                         mov     eax, esi
    .text:00414BD2 50                                            push    eax
    .text:00414BD3 FF 33                                         push    dword ptr [ebx]
    .text:00414BD5 FF 15 94 12 40 00                             call    ds:lstrcmpiW
    .text:00414BDB 85 C0                                         test    eax, eax
    .text:00414BDD 74 08                                         jz      short loc_414BE7
    .text:00414BDF 47                                            inc     edi
    .text:00414BE0 83 FF 28                                      cmp     edi, 28h
    .text:00414BE3 72 D8                                         jb      short loc_414BBD
    .text:00414BE5 EB 1F                                         jmp     short loc_414C06
    ...

In [15]:
match = re.search(rb'\x66\x8b\x04\xfd(?P<indexes_addr>[\s\S]{4})', pe.get_memory_mapped_image())
indexes_addr = struct.unpack("I", match.group("indexes_addr"))[0]

for i in range(40):
    index_addr = indexes_addr + i * 8
    index_rva = index_addr - pe.OPTIONAL_HEADER.ImageBase
    enc_string_index = pe.get_word_at_rva(index_rva)
    
    plain_string = decrypt_string(pe, enc_string_index)
    print(f"{enc_string_index}: {plain_string}")

227: os_shutdown
228: os_reboot
229: url_open
233: dns_filter_add
234: dns_filter_remove
230: bot_uninstall
231: bot_update
232: bot_transfer
235: bot_bc_add
236: bot_bc_remove
237: bot_httpinject_disable
238: bot_httpinject_enable
239: fs_path_get
240: fs_search_add
241: fs_search_remove
242: user_destroy
243: user_logoff
244: user_execute
245: user_cookies_get
246: user_cookies_remove
247: user_certs_get
248: user_certs_remove
249: user_url_block
250: user_url_unblock
251: user_homepage_set
252: user_ftpclients_get
253: user_emailclients_get
254: user_flashplayer_get
255: user_flashplayer_remove
256: module_execute_enable
257: module_execute_disable
258: module_download_enable
259: module_download_disable
260: info_get_software
261: info_get_antivirus
262: info_get_firewall
266: ddos_start
267: ddos_stop
268: webinjects_update
271: close_browsers
